# Build an end-to-end Langchain Applications with Azure Open AI

In [68]:
import os
from dotenv import load_dotenv
import openai
import langchain
from langchain import PromptTemplate
from langchain.llms import OpenAI

from langchain import OpenAI, SQLDatabase
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents import create_sql_agent
from langchain.embeddings import OpenAIEmbeddings
from langchain.utilities import SerpAPIWrapper
from langchain.agents import initialize_agent, AgentType, Tool
from langchain.llms import OpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.chains import LLMMathChain, LLMChain, RetrievalQA, SequentialChain, TransformChain
from langchain.vectorstores.azuresearch import AzureSearch
from langchain.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from pydantic import BaseModel, Field, validator
from langchain.output_parsers import PydanticOutputParser

In [28]:
# Azure
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_DEPLOYMENT_ENDPOINT = os.getenv("OPENAI_DEPLOYMENT_ENDPOINT")
OPENAI_DEPLOYMENT_NAME = os.getenv("OPENAI_DEPLOYMENT_NAME")
OPENAI_MODEL_NAME = os.getenv("OPENAI_MODEL_NAME")
OPENAI_API_VERSION = os.getenv("OPENAI_API_VERSION")
OPENAI_EMBEDDING_DEPLOYMENT_NAME = os.getenv("OPENAI_EMBEDDING_DEPLOYMENT_NAME")
OPENAI_EMBEDDING_MODEL_NAME = os.getenv("OPENAI_EMBEDDING_MODEL_NAME")
OPENAI_DEPLOYMENT_VERSION = os.getenv("OPENAI_DEPLOYMENT_VERSION")
OPENAI_EMBEDDING_VERSION = os.getenv("OPENAI_EMBEDDING_VERSION")

# SQL
database_user = os.getenv("DATABASE_USERNAME")
database_password = os.getenv("DATABASE_PASSWORD")
database_server = os.getenv("DATABASE_SERVER")
database_db = os.getenv("DATABASE_DB")

# SerpAPI
search = SerpAPIWrapper(serpapi_api_key = os.getenv("SERPER_API_KEY"))

# cognitive service
vector_store_address = os.getenv("VECTOR_STORE_ADDRESS")
vector_store_password = os.getenv("VECTOR_STORE_PASSWORD")

#init Azure OpenAI
openai.api_type = "azure"
openai.api_version = OPENAI_DEPLOYMENT_VERSION
openai.api_base = OPENAI_DEPLOYMENT_ENDPOINT
openai.api_key = OPENAI_API_KEY

In [42]:
llm = OpenAI(engine="text-davinci-003", temperature=0)
llm("Tell me a joke")

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


'\n\nQ: What did the fish say when it hit the wall?\nA: Dam!'

## 1. Querying Transactional datastores

In [101]:
connection_string = f"mssql+pymssql://{database_user}:{database_password}@{database_server}.database.windows.net:1433/{database_db}"
db = SQLDatabase.from_uri(connection_string)
toolkit = SQLDatabaseToolkit(db=db, llm=llm, reduce_k_below_max_tokens=True)
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True
)

## 2. Populate Vector DB with Embeddings

In [33]:
model = 'text-embedding-ada-002'
os.environ["OPENAI_API_VERSION"] = os.getenv("OPENAI_EMBEDDING_VERSION")
index_name: str = "langchain-vector-demo"
embeddings: OpenAIEmbeddings = OpenAIEmbeddings(deployment=model, chunk_size=1)
vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=vector_store_address,
    azure_search_key=vector_store_password,
    index_name=index_name,
    embedding_function=embeddings.embed_query,
)
loader = DirectoryLoader('./data', glob='*.pdf', show_progress=True)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)
list_of_docs = vector_store.add_documents(documents=docs)


100%|██████████| 3/3 [00:01<00:00,  2.83it/s]


['N2RiODgxYTAtYWUyNy00YmIxLWIwZWEtMjAwYjNlNGY3NDZl',
 'NzE2N2VmMjAtNTgwZS00MzlmLTlkNzItYTE5NjFkYzdhMTc2',
 'ZTRkMGUzMjgtMzVmYi00OGIwLWI5ZGUtZDdlMjM4NDZlYWU5',
 'NzY1ZWFjNTItNDYyYS00ZTc1LWFlOTAtOTBmYmMzMDNiMmI2',
 'ZDgxNTJmYTctNzA2Yy00ZTkyLWExZTUtN2M5ZGQzMGE2MDBj',
 'Y2E0ODJiNzMtZmRiMC00YWYyLTg4MjMtNTRkYjg4NTZjNjE1',
 'ZGZiMmViMmUtMTE5OS00ZTU1LThmODctOGEwZmM2YWI3Zjkz',
 'ZWFiMjQ5OWEtZWVmMC00YzkzLThmOGQtZDExMDdjYmM2MjI2',
 'M2Q4MGQ2ZjUtZjVhZS00M2QwLTljOGUtZDFkNTJmNzhmMTdj',
 'YmQ3Mzk3NDItODVhZi00ZjI5LTk4OTQtNGUzNDNhNGQ4OTJl',
 'MGJhODliMjYtOWEwYS00MjBiLTkyMmQtZTA3NGZjMjBhZWI3',
 'MjAzZDgwZTgtOGE5OC00MmJkLWFhMjctNGQxYjczZDdhOTQ4',
 'MTM1NGRjZWMtMjA4MC00M2MyLTk2M2QtYWRmOGZmNDJkZGFh',
 'MjdlMjQ0NTUtNjU5Zi00NWQyLThhYWItOTgyMzc2MWUyZjNm',
 'OTk5ZmY2NTgtZTVkOS00OTg3LThkOTQtMzUyZmNkYmJjNzVl',
 'MDg2MzhmMDYtNzgxNS00Yzc4LThkOGYtYzA1OGE5NTkyMDYx',
 'MzE3NTA1ZGQtMWFiYi00ZmM0LWFmMzQtMTI1MjU0YTdhZGYy',
 'YjM2NGUwMWMtNWE2NC00MDc0LWEwNDEtNjU1NWE4YzIyYzQ1',
 'ZmY2ZjY0MTctMWE0NS00ZjI2LThkY2ItYTc1YWI3YjMy

In [35]:
# Perform a similarity search
docs = vector_store.similarity_search(
    query="What are some good locations in bengaluru to visit?",
    k=3,
    search_type="similarity",
)

print(docs[0].page_content)

Located in the heart of city, set on a 300 acre area, Cubbon Park is a great place to laze around in the lush green paradise. It has a treasure of different species of exotic plants. The park was named after the erstwhile Commissioner of Bangalore, Sir Mark Cubbon. Many ornamental and flowering trees, both exotic and indigenous, can be found in the park. It's a popular place for walkers, joggers, and nature lovers. Bal Bhavan in the park is a great place for the kids to enjoy. An aquarium is also located in Cubbon Park, with many varieties of exotic and ornamental fish.

Ulsoor Lake


## 3. Lang Chain (Prompt Templates, Chaining, Agents)

In [126]:
class QueryInput(BaseModel):
    Source: str
    Destination: str
    Pax: int
    Date: str
    
parser = PydanticOutputParser(pydantic_object=QueryInput)

def parse_output(inputs: dict) -> dict:
    text = inputs["json_string"]
    d = parser.parse(text)
    return {"Source" : d.Source, "Destination" : d.Destination, "Pax": d.Pax, "Date": d.Date}

def run(user_query, llm, vector_store):
    
   prompt = """
   Extract source, destination, number of people as 'Pax' and date of travel as 'Date' from below provided text.
   Provide information as json
   text: {input}
   """
   
   promptEngine = PromptTemplate(
       input_variables=["input"],
       template=prompt       
   )
   
   parse_chain = LLMChain(llm=llm, prompt=promptEngine, output_key="json_string")
   
   transform_chain = TransformChain(input_variables=["json_string"], output_variables=["Source", "Destination", "Pax", "Date"],  transform=parse_output)
   
   # query database for available flights
   sql_template = """You are an expert trip adviser. \
       Give me a complete itinerary for the trip with the total price using cheapest flight, places to visit in the destination city.
       Find 2 flights from {Source} to {Destination}. Fine the cheapest flight. Find the total price for {Pax} passengers. \
       Find the weather in {Destination} on {Date} in degrees celsius. \
       Respond with complete itinerary in a list format for the trip with the total price using cheapest flight, weather in degrees celsius and top 5 places to visit in the destination city."""
   
   sql_prompt_template = PromptTemplate(input_variables=["Source", "Destination", 'Pax', 'Date'], template=sql_template)
   
   # get weather information using serp api.
   search = SerpAPIWrapper(serpapi_api_key = os.getenv("SERPER_API_KEY"))
   
   # get math information using LLM math chain
   llm_math = LLMMathChain.from_llm(llm=llm)
   
   # get information about places to visit using retrieval QA and vector store
   places_to_visit = RetrievalQA.from_chain_type(
       llm=llm, chain_type="stuff", retriever=vector_store.as_retriever()
   )
   
   tools = [
       Tool(
           name = "Search",
           func = search.run,
           description = '''useful for when you need to answer questions about current events,
               getting weather forecast infomation like temperature and humidity
               do not use this for searching flight information
               do not use this for searching information about places to visit
               '''
       ),
       Tool(
           name = 'Calculator',
           func = llm_math.run,
           description = 'Useful for when you need to answer questions about math. Use for converting from degrees celcius to fahrenheit'
       ),
       Tool(
           name = 'SQL',
           func = agent_executor.run,
           description='''Useful when you need to search flights using source, destination, pax only. 
           Use to find the cheapest flight. Use to find the total price for pax passengers.
           do not use this for searching information about places to visit           
           '''
       ),
       Tool(
           name = 'vector_store',
           func = places_to_visit.run,
           description = 'Useful when you need to find places to visit in a city'
       )
   ]
   
   agent = initialize_agent(
       tools,
       llm,
       agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
       verbose = True
   )
   
   overall_chain = SequentialChain(
       chains=[parse_chain, transform_chain],
       output_variables=["Source", "Destination", 'Pax', 'Date'],
       input_variables=["input"],
       verbose=True
   )
     
   inputs  = overall_chain(user_query)
   
   final_prompt = sql_prompt_template.format(Source=inputs["Source"], Destination = inputs["Destination"], Pax = inputs["Pax"], Date = inputs["Date"])
   
   print(f'final prompt: {final_prompt}')
   
   final_output = agent.run(final_prompt)
   
   return final_output
   
user_query = 'Two of my friends and I are travelling from Hyderabad to bangalore on 29th July, 2023.'
final_output = run(user_query, llm, vector_store)
   



> Entering new  chain...

> Finished chain.
final prompt: You are an expert trip adviser.        Give me a complete itinerary for the trip with the total price using cheapest flight, places to visit in the destination city.
       Find 2 flights from Hyderabad to Bangalore. Fine the cheapest flight. Find the total price for 3 passengers.        Find the weather in Bangalore on 29th July, 2023 in degrees celsius.        Respond with complete itinerary in a list format for the trip with the total price using cheapest flight, weather in degrees celsius and top 5 places to visit in the destination city.


> Entering new  chain...

Action:
```
{
  "action": "SQL",
  "action_input": "Find 2 flights from Hyderabad to Bangalore. Find the cheapest flight. Find the total price for 3 passengers."
}
```


> Entering new  chain...
Action: sql_db_list_tables
Action Input: 
Observation: flights
Thought: I should query the schema of the flights table.
Action: sql_db_schema
Action Input: flights
Obse